## Acerca del conjunto de datos

El analisis de personalidad de clientes es un analisis detallado de los clientes ideales de una empresa. Ayuda a la empresa a conocer mejor a sus clientes y les permite modificar los productos de acuerdo con las necesidades, comportmaientos e inquietudes de los diferentes tipos de clientes.

Ayuda a la empresa a modificar su producto en funcion de su cliente objetivo de distintos tipos de segmentos de clientes. Ejemplo, en lugar de gastar dinero para comercializar un nuevo producto a todos los clientes, una empresa puede analizar que segmento de clientes tienen mas probabilidades de comprar el producto y luego comercializarlo solo en ese segmento en particular

### Cargar librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from scipy.stats import boxcox, zscore
from scipy.special import inv_boxcox

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans

### Cargar Datos

In [2]:
df = pd.read_csv('./marketing_campaign.csv', sep='\t')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


### Entendiendo los datos

In [5]:
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

__Atributos de las personas__

- ID: Identificador unico del cliente.
- Year_Birth: Fecha de nacimento del cliente.
- Education: Nivel educativo.
- Marital_Status: Estado civil.
- Income: Ingresos familiares anuales del cliente.
- Kidhome: Numero de niños en el hogar del cliente.
- Teenhome: Numero de adolescentes en el hogar del cliente.
- Dt_Customer: Fecha de alta del cliente en la empresa.
- Recency: Numero de dias desde la ultima compra del cliente.
- Complain: 1 si el cliente se quejo en los ultimos dos años, 0 en caso contrario.

__Atributos del producto__

- MntWines: Cantidad gastada en vino en los ultimos dos años.
- MntFruits: Cantidad gastada en frutas en los utlimos dos años.
- MntMeatProduts: Cantidad gastada en carnes en los utlimos dos años.
- MntFishProduct: Cantidad gastada en pescado en los utlimos dos años.
- MntSweetProduct: Cantidad gastada en dulces en los ultimos dos años.
- MntGoldProds: Cantidad gastada en oro en los ultimos dos años. 

__Atributos de la promocion__

- NumDealsPurchases: Numero de compras realizadas con descuento.
- AcceptedCmp1: 1 si el cliente acepto la oferta en la primera campaña, 0 en caso contrario.
- AcceptedCmp2: 1 si el cliente acepto la oferta en la segunda campaña, 0 en caso contrario.
- AcceptedCmp3: 1 si el cliente acepto la oferta en la tercera campaña, 0 en caso contrario.
- AcceptedCmp2: 1 si el cliente acepto la oferta en la cuarta campaña, 0 en caso contrario.
- AcceptedCmp2: 1 si el cliente acepto la oferta en la quinta campaña, 0 en caso contrario.
- Response: 1 si el cliente acepto la oferta en la ultima campaña, 0 en caso contrario.

__Atributos del lugar__

- NumWebPurchases: Numero de compras realizadas a traves del sitio web de la empresa.
- NumCatalogPurchases: Numero de compras realizadas por el catalogo de la empresa.
- NumStorePurchases: Numero de compras realizadas directamente en la tienda.
- NumWebVisitsMonth: Numero de visitas al sitio web de la empresa en el ultimo mes.

__Objetivo__

Realizar una agrupacion para resumir y comprender los segmentos de clientes y de esta forma identificar grupos distintos dentro de la base de clientes.

### Analisis Estadistico descriptivo